In [1]:
import pandas as pd
df = pd.read_csv('heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dfle = df
dfle[['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope']] = dfle[['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope']].apply(le.fit_transform)
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1,140,289,0,1,172,0,0.0,2,0
1,49,0,2,160,180,0,1,156,0,1.0,1,1
2,37,1,1,130,283,0,2,98,0,0.0,2,0
3,48,0,0,138,214,0,1,108,1,1.5,1,1
4,54,1,2,150,195,0,1,122,0,0.0,2,0


In [9]:
upper_limit = df.RestingBP.mean() +3*df.RestingBP.std()
lower_limit = df.RestingBP.mean() -3*df.RestingBP.std()
df1 = df[(df.RestingBP<upper_limit)&(df.RestingBP>lower_limit)]
df1.shape

(910, 12)

In [11]:
upper_limit = df1.Cholesterol.mean() +3*df1.Cholesterol.std()
lower_limit = df1.Cholesterol.mean() -3*df1.Cholesterol.std()
df2 = df1[(df1.Cholesterol<upper_limit)&(df1.Cholesterol>lower_limit)]
df2.shape

(907, 12)

In [12]:
upper_limit = df2.MaxHR.mean() +3*df2.MaxHR.std()
lower_limit = df2.MaxHR.mean() -3*df2.MaxHR.std()
df3 = df2[(df2.MaxHR<upper_limit)&(df2.MaxHR>lower_limit)]
df3.shape

(906, 12)

In [14]:
from sklearn.preprocessing import StandardScaler
x = df3.drop('HeartDisease',axis=1)
y = df3.HeartDisease
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
x_scaled

array([[-1.43268734,  0.51513409,  0.22715383, ..., -0.8232726 ,
        -0.83125986,  1.04605981],
       [-0.47787471, -1.94124214,  1.27183082, ..., -0.8232726 ,
         0.10838011, -0.60503808],
       [-1.75095822,  0.51513409,  0.22715383, ..., -0.8232726 ,
        -0.83125986,  1.04605981],
       ...,
       [ 0.37084763,  0.51513409, -0.81752317, ...,  1.2146645 ,
         0.2963081 , -0.60503808],
       [ 0.37084763, -1.94124214,  0.22715383, ..., -0.8232726 ,
        -0.83125986, -0.60503808],
       [-1.64486793,  0.51513409,  1.27183082, ..., -0.8232726 ,
        -0.83125986,  1.04605981]])

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_scaled,y,stratify=y,random_state=10)

In [17]:
from sklearn.svm import SVC
model = SVC()
model.fit(x_train,y_train)

SVC()

In [18]:
model.score(x_test,y_test)

0.8766519823788547

In [19]:
from sklearn.ensemble import BaggingClassifier
bag_model = BaggingClassifier(
    estimator=SVC(),
    n_estimators=50,
    max_samples=0.8,
    oob_score=True,
    random_state=0
)
bag_model.fit(x_train,y_train)

BaggingClassifier(estimator=SVC(), max_samples=0.8, n_estimators=50,
                  oob_score=True, random_state=0)

In [20]:
bag_model.oob_score_

0.8630338733431517

In [21]:
bag_model.score(x_test,y_test)

0.8766519823788547

In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
score = cross_val_score(DecisionTreeClassifier(),x_scaled,y)
score.mean()

0.7449760184566813

In [23]:
bag_model = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=50,
    max_samples=0.8,
    oob_score=True,
    random_state=0
)
bag_model.fit(x_train,y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.8,
                  n_estimators=50, oob_score=True, random_state=0)

In [24]:
bag_model.oob_score_

0.8394698085419735

In [25]:
bag_model.score(x_test,y_test)

0.8766519823788547